# Simple Workflow

Welcome to the first tutorial for `janis`, where we'll untar a file, compile the contents and retar the result:

![title](simple.png)

In this first step, we're going to input all the data types and tools we require.

In [ ]:
import janis as j

# Data types - These help us logically connect workflows
from janis.unix.data_types.tarfile import TarFile

# Tools - The command line tools we're going to call
from janis.unix.tools.compile import Compile
from janis.unix.tools.tar import Tar
from janis.unix.tools.untar import Untar

In [2]:
w = j.WorkflowBuilder("simple")

Let's declare our inputs:

In [3]:
inp = w.input("tarFile", TarFile, value="s3://path/to/hello.tar")

Now our steps using the tools we imported

In [4]:
w.step("untar", Untar(tarFile=w.tarFile))
w.step("compile", Compile(file=w.untar.files), scatter="file")
w.step("tar", Tar(files=w.untar.files, files2=w.untar.files))

We want to collect our output from the retar step.

In [8]:
w.output("untarResult", source=w.untar.files)
w.output("out", source=w.tar.tarred)

In [11]:
wdl = w.translate("wdl", merge_resources=True)

2019-07-08T11:51:01 [WARN]: The command tool 'javaCompiler.compiled' used a star-bind (*) glob to find the output, but the return type was not an array. For WDL, the first element will be used, ie: 'glob("*.class")[0]'
=== WORKFLOW ===
version development

import "tools/Untar.wdl" as U
import "tools/javaCompiler.wdl" as J
import "tools/Tar.wdl" as T

workflow simple {
  input {
    File tarFile
  }
  call U.Untar as untar {
    input:
      tarFile=tarFile
  }
  scatter (f in untar.files) {
     call J.javaCompiler as compile {
      input:
        file=f
    }
  }
  call T.Tar as tar {
    input:
      files=untar.files,
      files2=compile.compiled
  }
  output {
    File out = tar.tarred
  }
}

=== TOOLS ===
:: tools/Untar.wdl ::
version development

task Untar {
  input {
    File tarFile
    Int? runtime_cpu
    Int? runtime_memory
  }
  command {
    tar xf \
       ${tarFile}
  }
  runtime {
    docker: "ubuntu:latest"
    cpu: runtime_cpu
    memory: if defined(runtime_memory)

In [10]:
cwl = w.translate("cwl")

=== WORKFLOW ===
class: Workflow
cwlVersion: v1.0
id: simple
inputs:
  tarFile:
    id: tarFile
    type: File
outputs:
  out:
    id: out
    outputSource: tar/tarred
    type: File
requirements:
  InlineJavascriptRequirement: {}
  ScatterFeatureRequirement: {}
  StepInputExpressionRequirement: {}
steps:
  compile:
    in:
      file:
        id: file
        source: untar/files
    out:
    - compiled
    run: tools/javaCompiler.cwl
    scatter:
    - file
  tar:
    in:
      files:
        id: files
        source: untar/files
      files2:
        id: files2
        source: compile/compiled
    out:
    - tarred
    run: tools/Tar.cwl
  untar:
    in:
      tarFile:
        id: tarFile
        source: tarFile
    out:
    - files
    run: tools/Untar.cwl


=== TOOLS ===
:: tools/Untar.cwl ::
baseCommand:
- tar
- xf
class: CommandLineTool
cwlVersion: v1.0
id: Untar
inputs:
- id: tarFile
  inputBinding:
    position: 0
  label: tarFile
  type: File
label: Untar
outputs:
- id: files
